## Import libraries

In [1]:
from pathlib import Path
import pandas as pd
import openpyxl
import math

## Create "Arrival_end_times" excel file

In [17]:
# Open Arrival_end_times excel file
Table_file1 = openpyxl.load_workbook("Arrival_end_times.xlsx")
arr_table = Table_file1.active

# Define new columns for end time data
arr_table.cell(row = 1, column = 5).value = "End_time"
arr_table.cell(row = 1, column = 6).value = "End_index"

## Open End_time excel file as a dataframe

In [18]:
end_df = pd.read_excel("End_time.xlsx")

In [6]:
end_df.head(5)

,Charge_mass,Standoff_distance,End_time,End_index
0,0.5,1,0.01104,1106
1,0.5,7,0.02850,2852
2,0.5,9,0.03447,3449
3,0.5,11,0.04046,4048
4,0.5,13,0.04644,4646


## Write end time data into Arrival_end_times excel file

In [19]:
for i in range(2, arr_table.max_row+1):
    for j in end_df.index:
        if arr_table.cell(row = i, column = 1).value == end_df["Charge_mass"][j]:
            if arr_table.cell(row = i, column = 2).value == end_df["Standoff_distance"][j]:
                arr_table.cell(row = i, column = 5). value = end_df["End_time"][j]
                arr_table.cell(row = i, column = 6). value = end_df["End_index"][j]
                end_df_T = end_df.T
                end_df_T.pop(j)
                end_df = end_df_T.T
                break
Table_file1.save("Arrival_end_times.xlsx")

## Open Change_time excel file as a dataframe

In [2]:
change_df = pd.read_excel("Change_time.xlsx")

In [3]:
change_df.head(5)

,Charge_mass,Standoff_distance,Change_time,Change_index
0,0.5,1,0.00130,132
1,0.5,7,0.01856,1858
2,0.5,9,0.02453,2455
3,0.5,11,0.03050,3052
4,0.5,13,0.03648,3650


## Open Arrival_end_times file

In [4]:
# Open Arrival_end_times excel file
Table_file2 = openpyxl.load_workbook("Arrival_end_times.xlsx")
arr_end_table = Table_file2.active

# Define new columns for end time data
arr_end_table.cell(row = 1, column = 7).value = "Change_time"
arr_end_table.cell(row = 1, column = 8).value = "Change_index"

## Write change time data into Arrival_end_change_times excel file

In [7]:
for i in range(2, arr_end_table.max_row+1):
    for j in change_df.index:
        if arr_end_table.cell(row = i, column = 1).value == change_df["Charge_mass"][j]:
            if arr_end_table.cell(row = i, column = 2).value == change_df["Standoff_distance"][j]:
                arr_end_table.cell(row = i, column = 7). value = change_df["Change_time"][j]
                arr_end_table.cell(row = i, column = 8). value = change_df["Change_index"][j]
                change_df_T = change_df.T
                change_df_T.pop(j)
                change_df = change_df_T.T
                break
Table_file2.save("Arrival_end_change_times.xlsx")

## Access input and output files

In [8]:
#Access input folder
input_dir1 = Path ("Pressure_curves_excel_with_MinTime")
print ("input",input_dir1)

#Access output directory
output_dir1 = Path ("Pressure_profiles")
output_dir1.mkdir(parents = True, exist_ok= True)
print ("output",output_dir1)

input Pressure_curves_excel_with_MinTime
output Pressure_profiles


## p_profile function to filter required pressure profile

In [10]:
def p_profile (file_path, arrival_index, end_index, change_time):
    excel_file = openpyxl.load_workbook(file_path)
    excel_sheet = excel_file.active
    
    p_time_history = []
    
    for row in excel_sheet.iter_rows(min_row=arrival_index,
                                     max_row=end_index,
                                     min_col=1, max_col=2,
                                     values_only=True):
        if row[0] < change_time:
            positive = "positive" # Replace with the value you want to add to the new column
            p_row = row + (positive,)
            p_time_history.append (p_row)
        else:
            negative = "negative"
            n_row = row + (negative,)
            p_time_history.append (n_row)
    
    return p_time_history

## Create pressure profile excel file

In [ ]:
# Open file with arrival and end indexes
Table_file3 = openpyxl.load_workbook("Arrival_end_change_times.xlsx")
arr_end_change = Table_file3.active

# Create empty summary dataframe
summary = pd.DataFrame(columns = ["Folder_name", "File_name", "Datapoints_count"])

for k in range(2, arr_end_change.max_row+1):
    
    # Set folder name
    Charge_mass = arr_end_change.cell (row = k, column = 1).value
    fol_name = "L13_"+ str(int(Charge_mass*2))
    print (fol_name)
    
    # Find folder name in input_dir1
    input_dir2=next(input_dir1.glob(fol_name))
    print (input_dir2)
    
    # Create or find output folder name in output_dir1
    output_dir2 = Path(output_dir1/fol_name)
    output_dir2.mkdir(parents = True, exist_ok= True)
    
    # Set file name
    Standoff_dis = arr_end_change.cell (row = k, column = 2).value
    fil_name = "P"+str(int(Standoff_dis*2-1))+".xlsx"
    print (fil_name)
    
    # Find file name in folder
    file_path = next(input_dir2.glob(fil_name))
    print ("file_path", file_path)
    
    # Get arrival and end indexes
    arrival_index = arr_end_change.cell (row = k, column = 4).value
    end_index = arr_end_change.cell (row = k, column = 6).value
    change_index = arr_end_change.cell (row = k, column = 8).value
    
    arrival_time = arr_end_change.cell (row = k, column = 3).value
    end_time = arr_end_change.cell (row = k, column = 5).value
    change_time = arr_end_change.cell (row = k, column = 7).value
    
    print ("arrival_time", arrival_time)
    print ("end_time", end_time)
    
    # Get required pressure profile
    pressure_profile = p_profile (file_path, arrival_index, end_index, change_time)
    
    # Create dataframe with all data
    arr_end_df = pd.DataFrame( pressure_profile, columns = ["Time", "Pressure", "Phase"])
    Charge_mass_column = [Charge_mass] * len(arr_end_df)
    Standoff_dis_column = [Standoff_dis] * len(arr_end_df)
    arr_end_df.insert(0, "Charge_mass", Charge_mass_column)
    arr_end_df.insert(1, "Standoff_distance", Standoff_dis_column)
    
    # Append summary dataframe
    dt_details = {"Folder_name" : [fol_name],
                  "File_name" : [fil_name],
                  "Datapoints_count" : [len(arr_end_df)]}
    dt_details_row = pd.DataFrame(dt_details)
    summary = pd.concat([summary, dt_details_row], ignore_index = True)
    
    # Write pressure profile to a new excel file
    excel_file = output_dir2/f"{fil_name}"
    with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
        arr_end_df.to_excel(writer, sheet_name = 'Sheet1', index = False, header = True)

L13_1
Pressure_curves_excel_with_MinTime\L13_1
P1.xlsx
file_path Pressure_curves_excel_with_MinTime\L13_1\P1.xlsx
arrival_time 0.00065976753831
end_time 0.011039755307
L13_1
Pressure_curves_excel_with_MinTime\L13_1
P13.xlsx
file_path Pressure_curves_excel_with_MinTime\L13_1\P13.xlsx
arrival_time 0.015829600394
end_time 0.028499947861
L13_1
Pressure_curves_excel_with_MinTime\L13_1
P17.xlsx
file_path Pressure_curves_excel_with_MinTime\L13_1\P17.xlsx
arrival_time 0.021559774876
end_time 0.034469928592
L13_1
Pressure_curves_excel_with_MinTime\L13_1
P21.xlsx
file_path Pressure_curves_excel_with_MinTime\L13_1\P21.xlsx
arrival_time 0.027349945158
end_time 0.040459807962
L13_1
Pressure_curves_excel_with_MinTime\L13_1
P25.xlsx
file_path Pressure_curves_excel_with_MinTime\L13_1\P25.xlsx
arrival_time 0.033179905266
end_time 0.046439867467
L13_1
Pressure_curves_excel_with_MinTime\L13_1
P29.xlsx
file_path Pressure_curves_excel_with_MinTime\L13_1\P29.xlsx
arrival_time 0.039039634168
end_time 0.05242

## Create Pressure_profile_summary file 

In [23]:
with pd.ExcelWriter("Pressure_profile_summary.xlsx", engine='openpyxl') as writer:
    summary.to_excel(writer, sheet_name = 'Sheet1', index = False, header = True)